In [5]:
import matplotlib.pyplot as plt
import cv2
import json
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as transforms
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader, TensorDataset, Sampler
from tqdm import tqdm
from torchvision.transforms import Compose, RandomHorizontalFlip, RandomResizedCrop, ColorJitter, ToTensor, Normalize
import pandas as pd
from progressbar import ProgressBar 
import time
from multiprocessing import Pool
from functools import partial

from sklearn.model_selection import train_test_split
from plots import plot_val_3, plot_val_4, plot_loops
from torchsummary import summary
from einops.layers.torch import Rearrange
import csv
from torchsummary import summary

from torch.utils.data import Dataset, DataLoader, Sampler

#original seed
torch.manual_seed(42)
np.random.seed(42)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad) 

class ClipOutput(nn.Module):
    def forward(self, x):
        return torch.clamp(x,min=0,max=1)
    
def seed_function():
    seed = 42
    torch.manual_seed(seed)
    np.random.seed(seed)


ModuleNotFoundError: No module named 'plots'

In [6]:
class ResidualBlock_increasing(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels//4, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels//4)
        self.conv2 = nn.Conv2d(out_channels//4, out_channels//2, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels//2)

        
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels)
        )
        
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        residual = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        
        shortcut = self.shortcut(residual)
        out += shortcut
        out = self.relu(out)
        
        return out
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        identity = self.shortcut(identity)
        out += identity
        out = self.relu(out)
        
        return out
class ResidualConvUnit(nn.Module):
    """Residual convolution module."""

    def __init__(self, features):
        """Init.
        Args:
            features (int): number of features
        """
        super().__init__()

        self.conv1 = nn.Conv2d(
            features, features, kernel_size=3, stride=1, padding=1, bias=True
        )

        self.conv2 = nn.Conv2d(
            features, features, kernel_size=3, stride=1, padding=1, bias=True
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        """Forward pass.
        Args:
            x (tensor): input
        Returns:
            tensor: output
        """
        out = self.relu(x)
        out = self.conv1(out)
        out = self.relu(out)
        out = self.conv2(out)

        return out + x
class FeatureFusionBlock(nn.Module):
    """Feature fusion block."""

    def __init__(self, features):
        """Init.
        Args:
            features (int): number of features
        """
        super(FeatureFusionBlock, self).__init__()

        self.resConfUnit1 = ResidualConvUnit(features)
        self.resConfUnit2 = ResidualConvUnit(features)

    def forward(self, *xs):
        """Forward pass.
        Returns:
            tensor: output
        """  
        output = xs[0]

        if len(xs) == 2:
            output += self.resConfUnit1(xs[1])

        output = self.resConfUnit2(output)

        output = nn.functional.interpolate(
            output, scale_factor=2, mode="bilinear", align_corners=True
        )

        return output



In [7]:
class ViT_fusion_4_heads(nn.Module):
    def __init__(self, patch_size = 16, hidden_dim=512, num_heads=4, num_layers=16):
        super().__init__()
        
        
        self.fusionblock4 = FeatureFusionBlock(256)
        self.fusionblock3 = FeatureFusionBlock(256)
        self.fusionblock2 = FeatureFusionBlock(256)
        self.fusionblock1 = FeatureFusionBlock(256)
        
        self.head = nn.Sequential(

            nn.Conv2d(256, 256//2, kernel_size = 3, stride=1, padding=1),
            nn.Upsample(scale_factor=2, mode="bilinear"),
            nn.Conv2d(256//2, 32, kernel_size = 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(32, 1, kernel_size = 1, stride = 1, padding= 0),
            nn.ReLU(True),
            nn.Identity()
    
        )
        
        self.hidden_dim = hidden_dim
        self.num_patches = (256 // patch_size)**2
        self.proj1 = nn.Conv2d(3, hidden_dim, kernel_size=patch_size, stride=patch_size)
        
        
        self.resample1 = nn.Sequential(
            nn.Conv2d(hidden_dim, 256, kernel_size = 1),
            nn.ConvTranspose2d(256, 256, kernel_size=4, stride=4, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256)
        )
        
        self.resample2 = nn.Sequential(
            nn.Conv2d(hidden_dim, 256, kernel_size = 1),
            nn.ConvTranspose2d(256, 256, kernel_size=2, stride=2, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256)
        )

        self.resample3 = nn.Sequential(
            nn.Conv2d(hidden_dim, 256, kernel_size = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256)
        )
        
        self.resample4 = nn.Sequential(
            nn.Conv2d(hidden_dim, 256, kernel_size = 1),
            nn.Conv2d(256, 256, kernel_size = 2, stride = 2, padding= 0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256)
        )
        
       
        # Transformer encoder layers
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_dim, num_heads, dim_feedforward=hidden_dim*4),
            num_layers=num_layers
        )
        
        
        
        self.flatten = Rearrange('b c h w -> b (h w) c')
        self.positional_encoding1 = nn.Parameter(torch.zeros(1, self.num_patches, hidden_dim))
       
        
        
    
    def forward(self, x):
        intermediate_layers = [4,8,12,16]
        output_intermediate_layers = []
        
        #32x3x256x256 -> 32x512x64x64
        #print("pre_proj1",x.shape)
        x = self.proj1(x)
        #print("post_proj1",x.shape)
        x = self.flatten(x)
        #print("pre_positional",x.shape)
        x = x + self.positional_encoding1
        #print("post_positional",x.shape)
        
        #transformer1
        x = x.transpose(0,1)
        for i, layer in enumerate(self.transformer.layers,1):
            x = layer(x)
            if i in intermediate_layers:
                output_intermediate_layers.append(x)
        
        x1 = output_intermediate_layers[0].transpose(0,1)
        x2 = output_intermediate_layers[1].transpose(0,1)
        x3 = output_intermediate_layers[2].transpose(0,1)
        x4 = output_intermediate_layers[3].transpose(0,1)
            
        x1 = x1.view(-1,16,16,512)
        x2 = x2.view(-1,16,16,512)
        x3 = x3.view(-1,16,16,512)
        x4 = x4.view(-1,16,16,512)
        
        x1 = self.resample1(x1.permute(0,3,1,2))
        x2 = self.resample2(x2.permute(0,3,1,2))
        x3 = self.resample3(x3.permute(0,3,1,2))
        x4 = self.resample4(x4.permute(0,3,1,2))
        
        path_4 = self.fusionblock4(x4)
        path_3 = self.fusionblock3(path_4, x3)
        path_2 = self.fusionblock2(path_3, x2)
        path_1 = self.fusionblock1(path_2, x1)
        
        
        
        output = self.head(path_1)
       
        
        return output.squeeze(1)
m = ViT_fusion_4_heads()   
count_parameters(m)

NameError: name 'Rearrange' is not defined

In [8]:
class ViT_fusion_8_heads(nn.Module):
    def __init__(self, patch_size = 16, hidden_dim=512, num_heads=8, num_layers=16):
        super().__init__()
        
        
        self.fusionblock4 = FeatureFusionBlock(256)
        self.fusionblock3 = FeatureFusionBlock(256)
        self.fusionblock2 = FeatureFusionBlock(256)
        self.fusionblock1 = FeatureFusionBlock(256)
        
        self.head = nn.Sequential(

            nn.Conv2d(256, 256//2, kernel_size = 3, stride=1, padding=1),
            nn.Upsample(scale_factor=2, mode="bilinear"),
            nn.Conv2d(256//2, 32, kernel_size = 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(32, 1, kernel_size = 1, stride = 1, padding= 0),
            nn.ReLU(True),
            nn.Identity()
    
        )
        
        self.hidden_dim = hidden_dim
        self.num_patches = (256 // patch_size)**2
        self.proj1 = nn.Conv2d(3, hidden_dim, kernel_size=patch_size, stride=patch_size)
        
        
        self.resample1 = nn.Sequential(
            nn.Conv2d(hidden_dim, 256, kernel_size = 1),
            nn.ConvTranspose2d(256, 256, kernel_size=4, stride=4, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256)
        )
        
        self.resample2 = nn.Sequential(
            nn.Conv2d(hidden_dim, 256, kernel_size = 1),
            nn.ConvTranspose2d(256, 256, kernel_size=2, stride=2, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256)
        )

        self.resample3 = nn.Sequential(
            nn.Conv2d(hidden_dim, 256, kernel_size = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256)
        )
        
        self.resample4 = nn.Sequential(
            nn.Conv2d(hidden_dim, 256, kernel_size = 1),
            nn.Conv2d(256, 256, kernel_size = 2, stride = 2, padding= 0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256)
        )
        
       
        # Transformer encoder layers
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_dim, num_heads, dim_feedforward=hidden_dim*4),
            num_layers=num_layers
        )
        
        
        
        self.flatten = Rearrange('b c h w -> b (h w) c')
        self.positional_encoding1 = nn.Parameter(torch.zeros(1, self.num_patches, hidden_dim))
       
        
        
    
    def forward(self, x):
        intermediate_layers = [4,8,12,16]
        output_intermediate_layers = []
        
        #32x3x256x256 -> 32x512x64x64
        #print("pre_proj1",x.shape)
        x = self.proj1(x)
        #print("post_proj1",x.shape)
        x = self.flatten(x)
        #print("pre_positional",x.shape)
        x = x + self.positional_encoding1
        #print("post_positional",x.shape)
        
        #transformer1
        x = x.transpose(0,1)
        for i, layer in enumerate(self.transformer.layers,1):
            x = layer(x)
            if i in intermediate_layers:
                output_intermediate_layers.append(x)
        
        x1 = output_intermediate_layers[0].transpose(0,1)
        x2 = output_intermediate_layers[1].transpose(0,1)
        x3 = output_intermediate_layers[2].transpose(0,1)
        x4 = output_intermediate_layers[3].transpose(0,1)
            
        x1 = x1.view(-1,16,16,512)
        x2 = x2.view(-1,16,16,512)
        x3 = x3.view(-1,16,16,512)
        x4 = x4.view(-1,16,16,512)
        
        x1 = self.resample1(x1.permute(0,3,1,2))
        x2 = self.resample2(x2.permute(0,3,1,2))
        x3 = self.resample3(x3.permute(0,3,1,2))
        x4 = self.resample4(x4.permute(0,3,1,2))
        
        path_4 = self.fusionblock4(x4)
        path_3 = self.fusionblock3(path_4, x3)
        path_2 = self.fusionblock2(path_3, x2)
        path_1 = self.fusionblock1(path_2, x1)
        
        
        
        output = self.head(path_1)
       
        
        return output.squeeze(1)

In [9]:
class ViT_fusion_16_heads(nn.Module):
    def __init__(self, patch_size = 16, hidden_dim=512, num_heads=16, num_layers=16):
        super().__init__()
        
        
        self.fusionblock4 = FeatureFusionBlock(256)
        self.fusionblock3 = FeatureFusionBlock(256)
        self.fusionblock2 = FeatureFusionBlock(256)
        self.fusionblock1 = FeatureFusionBlock(256)
        
        self.head = nn.Sequential(

            nn.Conv2d(256, 256//2, kernel_size = 3, stride=1, padding=1),
            nn.Upsample(scale_factor=2, mode="bilinear"),
            nn.Conv2d(256//2, 32, kernel_size = 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(32, 1, kernel_size = 1, stride = 1, padding= 0),
            nn.ReLU(True),
            nn.Identity()
    
        )
        
        self.hidden_dim = hidden_dim
        self.num_patches = (256 // patch_size)**2
        self.proj1 = nn.Conv2d(3, hidden_dim, kernel_size=patch_size, stride=patch_size)
        
        
        self.resample1 = nn.Sequential(
            nn.Conv2d(hidden_dim, 256, kernel_size = 1),
            nn.ConvTranspose2d(256, 256, kernel_size=4, stride=4, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            
        )
        
        self.resample2 = nn.Sequential(
            nn.Conv2d(hidden_dim, 256, kernel_size = 1),
            nn.ConvTranspose2d(256, 256, kernel_size=2, stride=2, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            
        )

        self.resample3 = nn.Sequential(
            nn.Conv2d(hidden_dim, 256, kernel_size = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            
        )
        
        self.resample4 = nn.Sequential(
            nn.Conv2d(hidden_dim, 256, kernel_size = 1),
            nn.Conv2d(256, 256, kernel_size = 2, stride = 2, padding= 0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            
        )
        
       
        # Transformer encoder layers
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_dim, num_heads, dim_feedforward=hidden_dim*4),
            num_layers=num_layers
        )
        
        
        
        self.flatten = Rearrange('b c h w -> b (h w) c')
        self.positional_encoding1 = nn.Parameter(torch.zeros(1, self.num_patches, hidden_dim))
       
        
        
    
    def forward(self, x):
        intermediate_layers = [4,8,12,16]
        output_intermediate_layers = []
        
        #32x3x256x256 -> 32x512x64x64
        print(self.positional_encoding1.shape)
        print("pre_proj1",x.shape)
        x = self.proj1(x)
        print("post_proj1",x.shape)
        x = self.flatten(x)
        print("pre_positional",x.shape)
        x = x + self.positional_encoding1
        print("post_positional",x.shape)
        
        print("STOPPP\n")
        #transformer1
        x = x.transpose(0,1)
        for i, layer in enumerate(self.transformer.layers,1):
            x = layer(x)
            if i in intermediate_layers:
                output_intermediate_layers.append(x)
        
        x1 = output_intermediate_layers[0].transpose(0,1)
        x2 = output_intermediate_layers[1].transpose(0,1)
        x3 = output_intermediate_layers[2].transpose(0,1)
        x4 = output_intermediate_layers[3].transpose(0,1)
            
        x1 = x1.view(-1,16,16,512)
        x2 = x2.view(-1,16,16,512)
        x3 = x3.view(-1,16,16,512)
        x4 = x4.view(-1,16,16,512)
        
        x1 = self.resample1(x1.permute(0,3,1,2))
        x2 = self.resample2(x2.permute(0,3,1,2))
        x3 = self.resample3(x3.permute(0,3,1,2))
        x4 = self.resample4(x4.permute(0,3,1,2))
        
        path_4 = self.fusionblock4(x4)
        path_3 = self.fusionblock3(path_4, x3)
        path_2 = self.fusionblock2(path_3, x2)
        path_1 = self.fusionblock1(path_2, x1)
        
        
        
        output = self.head(path_1)
       
        
        return output.squeeze(1)

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Encoder
        self.enc_conv1 = self._conv_block(3,96)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.enc_conv2 = self._conv_block(96,192)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.enc_conv3 = self._conv_block(192,384)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.enc_conv4 = self._conv_block(384,768)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Bottleneck
        self.bottleneck = self._conv_block(768,1536)

        # Decoder
        self.upconv4 = nn.ConvTranspose2d(1536, 768, kernel_size=2, stride=2)
        self.dec_conv4 = self._conv_block(1536, 768)

        self.upconv3 = nn.ConvTranspose2d(768, 384, kernel_size=2, stride=2)
        self.dec_conv3 = self._conv_block(768, 384)
        
        self.upconv2 = nn.ConvTranspose2d(384, 192, kernel_size=2, stride=2)
        self.dec_conv2 = self._conv_block(384, 192)

        self.upconv1 = nn.ConvTranspose2d(192, 96, kernel_size=2, stride=2)
        self.dec_conv1 = self._conv_block(192, 96)

        # Output layer
        self.output = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(96, 1, kernel_size=1, stride=1, padding=0),
            nn.ReLU()
        )
           
            
        
        #nn.init.kaiming_normal_(self.output.weight, nonlinearity="relu")
        
        
    def _conv_block(self, in_channels, out_channels):
        conv_block = nn.Sequential(
            nn.Conv2d(in_channels,out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels,out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
            
        )
        #for layer in conv_block:
            #if isinstance(layer, nn.Conv2d):
                #nn.init.kaiming_normal_(layer.weight, nonlinearity="relu")
        return conv_block
        
        
        
    def forward(self, x):
        # Encoder
       # print(x.shape)
        enc1 = self.enc_conv1(x)
        #print(enc1.shape)
        pool1 = self.pool1(enc1)
        #print(pool1.shape)

        enc2 = self.enc_conv2(pool1)
        pool2 = self.pool2(enc2)

        enc3 = self.enc_conv3(pool2)
        pool3 = self.pool3(enc3)

        enc4 = self.enc_conv4(pool3)
        pool4 = self.pool4(enc4)

        # Bottleneck
        bottleneck = self.bottleneck(pool4)
        #print(bottleneck.shape)

        # Decoder
        upconv4 = self.upconv4(bottleneck)
        merge4 = torch.cat([enc4, upconv4], dim=1)
       # print(merge4.shape)
        dec4 = self.dec_conv4(merge4)

        upconv3 = self.upconv3(dec4)
        merge3 = torch.cat([enc3, upconv3], dim=1)
        dec3 = self.dec_conv3(merge3)

        upconv2 = self.upconv2(dec3)
        merge2 = torch.cat([enc2, upconv2], dim=1)
        dec2 = self.dec_conv2(merge2)

        upconv1 = self.upconv1(dec2)
        merge1 = torch.cat([enc1, upconv1], dim=1)
        dec1 = self.dec_conv1(merge1)

        # Output layer
        output = self.output(dec1)
        
        #print(output.shape)
        return output.squeeze(1)
    
c = CNN()

In [11]:
class SeededShuffleSampler(Sampler):
    def __init__(self, data_source, seed):
        self.data_source = data_source
        self.seed = seed

    def __iter__(self):
        # Use torch.Generator() to create a new generator
        g = torch.Generator()
        g.manual_seed(self.seed)
        return iter(torch.randperm(len(self.data_source), generator=g).tolist())

    def __len__(self):
        return len(self.data_source)


class MyDataset(Dataset):
    def __init__(self, X, y, transform=None, epoch=0, global_seed=42):
        self.X = X
        self.y = y
        self.transform = transform
        self.epoch = epoch
        self.global_seed=global_seed
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        #x = torch.Tensor(self.features[idx])
        #y = torch.Tensor(self.targets[idx])
        x = self.X[idx]
        y = self.y[idx]
        
        if self.transform:
            rng_state = torch.get_rng_state()

            
            seed = self.global_seed + self.epoch
            torch.manual_seed(seed)
            x = self.transform(x)


            torch.manual_seed(seed)
            y = self.transform(y)

            torch.set_rng_state(rng_state)

        return x,y

In [12]:
transform_with_crop = Compose([
    RandomHorizontalFlip(0.5),
    RandomResizedCrop(size=(256,256), scale=(0.8,1.2), antialias=True)
])
transform_no_crop = Compose([
    RandomHorizontalFlip(0.5)
])

In [14]:
def get_data(size,choice):
    
    
    size = size.lower()
    
  
        
    X_train = torch.load(f"C:/Users/frede/Desktop/Bachelorprojekt/{choice}/{size}/X_train.pt")
    X_val = torch.load(f"C:/Users/frede/Desktop/Bachelorprojekt/{choice}/{size}/X_val.pt")
    X_test_VALID = torch.load(f"C:/Users/frede/Desktop/Bachelorprojekt/{choice}/{size}/X_test_VALID.pt")
    X_test_MIDAIR = torch.load(f"C:/Users/frede/Desktop/Bachelorprojekt/{choice}/{size}/X_test_MIDAIR.pt")

    y_train = torch.load(f"C:/Users/frede/Desktop/Bachelorprojekt/{choice}/{size}/y_train.pt")
    y_val = torch.load(f"C:/Users/frede/Desktop/Bachelorprojekt/{choice}/{size}/y_val.pt")
    y_test_VALID = torch.load(f"C:/Users/frede/Desktop/Bachelorprojekt/{choice}/{size}/y_test_VALID.pt")
    y_test_MIDAIR = torch.load(f"C:/Users/frede/Desktop/Bachelorprojekt/{choice}/{size}/y_test_MIDAIR.pt")

    
        
    return (X_train,y_train), (X_val, y_val), (X_test_VALID, y_test_VALID), (X_test_MIDAIR, y_test_MIDAIR)
    
choice1 = "Normalized_data_80"
choice2 = "Normalized_standardized_data_80"
choice3 = "Standardized_data_80"
choice4 = "Normalized_standardized_data_80_new"
data = get_data("xsmall", choice1)
#training_data = MyDataset(data[0][0].float(),data[0][1].float(),transform=transform) 
X_train, y_train = data[0][0].float(), data[0][1].float()
X_val, y_val = data[1][0].float(), data[1][1].float()
X_test_VALID, y_test_VALID = data[2][0].float(), data[2][1].float()
X_test_MIDAIR, y_test_MIDAIR = data[3][0].float(), data[3][1].float()

minimum_depth_train = torch.min(y_train)
maximum_depth_train = torch.max(y_train)
minimum_depth_val = torch.min(y_val)
maximum_depth_val = torch.max(y_val)

In [15]:
def make_csv(test_bool,train_loss, test_loss, settings, model_type, run_time, file_name):
    
    run_time = [run_time]
    if test_bool:
        epochs = train_loss
        valid_absrel_loss, valid_mae_loss, valid_delta1_loss, valid_delta2_loss, valid_delta3_loss, midair_abs_loss, midair_mae_loss, midair_delta1_loss, midair_delta2_loss, midair_delta3_loss = settings
    else:
        data_size,epochs,batch_size,lr,l2_reg,dropout_bool,hidden_layers,nheads,encoder_layers = settings

    if len(test_loss) > 1:
        test_loss1, test_loss2 = test_loss[0], test_loss[1]
        data = [test_loss1, valid_absrel_loss, valid_mae_loss, valid_delta1_loss, valid_delta2_loss, valid_delta3_loss, test_loss2,midair_abs_loss, midair_mae_loss, midair_delta1_loss, midair_delta2_loss, midair_delta3_loss,run_time]
    else:
        test_loss = test_loss[0]
        data = [train_loss, test_loss, data_size, epochs, run_time, batch_size, lr, l2_reg, dropout_bool, hidden_layers, nheads, encoder_layers]

    #print(data)
    
    
    if epochs == [5]:
        path = "5epochs"
    elif epochs == [25]:
        path = "25epochs"
    elif epochs == [50]:
        path = "50epochs"
    elif epochs == [100]:
        path = "100epochs"
    else:
        path = "300epochs"
    
    if model_type.lower() == "cnn":
        final_path = os.path.join(path, "cnns/csv", file_name)
    else:
        final_path = os.path.join(path, "vits/csv", file_name)
    
    
    
    with open(final_path, "w", newline='') as file_name:
        writer = csv.writer(file_name)
        for row in data:
            writer.writerow(row)
        

In [16]:
def save_weights(model, model_type, epochs, file_name,datasize):
    
    print(epochs)
    if epochs == 5:
        path = "5epochs"
    elif epochs == 25:
        path = "25epochs"
    elif epochs == 50:
        path = "50epochs"
    elif epochs == 100:
        path = "100epochs"
    else:
        path = "300epochs"
    
    if model_type.lower() == "cnn":
        new_path = os.path.join(path, "cnns/weights",datasize)
        if not os.path.exists(new_path):
            os.makedirs(new_path)
            final_path = os.path.join(new_path, file_name)
            torch.save(model.state_dict(), final_path)
        else:
            final_path = os.path.join(new_path, file_name)
            torch.save(model.state_dict(), final_path)
            
    else:
        new_path = os.path.join(path, "vits/weights",datasize)
        #print(new_path)
        if not os.path.exists(new_path):
            os.makedirs(new_path)
            final_path = os.path.join(new_path, file_name)
            torch.save(model.state_dict(), final_path)
        else:
            final_path = os.path.join(new_path, file_name)
            torch.save(model.state_dict(), final_path)
    
    
    
def norm(meter,min_val=0.1133,max_val=50.0):
    return((meter-min_val)/(max_val-min_val))

def reverse_norm(norm_val,min_val=0.1133,max_val=80.0):
    return(norm_val*(max_val-min_val)+min_val)

In [17]:
class WeightedRMSELoss1x(nn.Module):
    def __init__(self,min_dist=0.1133, max_dist=2.0, weight=1, min_depth=0.1133, max_depth=80):
        super(WeightedRMSELoss1x,self).__init__()
        self.min_dist = (min_dist-min_depth)/(max_depth-min_depth)
        self.max_dist = (max_dist-min_depth)/(max_depth-min_depth)
        self.weight = weight
        
    def forward(self,y_pred,y_true):
        squared_error = (y_pred - y_true)**2
        
        #creating a mask for depth values in the interval [0.1133, 2.0]
        critical_mask = (y_true >= self.min_dist) & (y_true <= self.max_dist)
        
        weighted_squared_error = torch.where(critical_mask, squared_error*self.weight, squared_error)
        
        rmse = torch.sqrt(torch.mean(weighted_squared_error))
        return rmse

class WeightedRMSELoss2x(nn.Module):
    def __init__(self,min_dist=0.1133, max_dist=2.0, weight=2, min_depth=0.1133, max_depth=80):
        super(WeightedRMSELoss2x,self).__init__()
        self.min_dist = (min_dist-min_depth)/(max_depth-min_depth)
        self.max_dist = (max_dist-min_depth)/(max_depth-min_depth)
        self.weight = weight
        
    def forward(self,y_pred,y_true):
        squared_error = (y_pred - y_true)**2
        
        #creating a mask for depth values in the interval [0.1133, 2.0]
        critical_mask = (y_true >= self.min_dist) & (y_true <= self.max_dist)
        
        weighted_squared_error = torch.where(critical_mask, squared_error*self.weight, squared_error)
        
        rmse = torch.sqrt(torch.mean(weighted_squared_error))
        return rmse
    
class WeightedRMSELoss5x(nn.Module):
    def __init__(self,min_dist=0.1133, max_dist=2.0, weight=5, min_depth=0.1133, max_depth=80):
        super(WeightedRMSELoss5x,self).__init__()
        self.min_dist = (min_dist-min_depth)/(max_depth-min_depth)
        self.max_dist = (max_dist-min_depth)/(max_depth-min_depth)
        self.weight = weight
        
    def forward(self,y_pred,y_true):
        squared_error = (y_pred - y_true)**2
        
        #creating a mask for depth values in the interval [0.1133, 2.0]
        critical_mask = (y_true >= self.min_dist) & (y_true <= self.max_dist)
        
        weighted_squared_error = torch.where(critical_mask, squared_error*self.weight, squared_error)
        
        rmse = torch.sqrt(torch.mean(weighted_squared_error))
        return rmse   
class WeightedRMSELoss10x(nn.Module):
    def __init__(self,min_dist=0.1133, max_dist=2.0, weight=10, min_depth=0.1133, max_depth=80):
        super(WeightedRMSELoss10x,self).__init__()
        self.min_dist = (min_dist-min_depth)/(max_depth-min_depth)
        self.max_dist = (max_dist-min_depth)/(max_depth-min_depth)
        self.weight = weight
        
    def forward(self,y_pred,y_true):
        squared_error = (y_pred - y_true)**2
        
        #creating a mask for depth values in the interval [0.1133, 2.0]
        critical_mask = (y_true >= self.min_dist) & (y_true <= self.max_dist)
        
        weighted_squared_error = torch.where(critical_mask, squared_error*self.weight, squared_error)
        
        rmse = torch.sqrt(torch.mean(weighted_squared_error))
        return rmse      

class ClippedRMSELoss(nn.Module):
    def __init__(self, min_value, max_value, penalty_weight):
        super(ClippedRMSELoss,self).__init__()
        self.min_value = min_value
        self.max_value = max_value
        self.penalty_weight = penalty_weight
        
    def forward(self,predictions, targets):
        
        rmse_loss = torch.sqrt(torch.mean((predictions-targets)**2))
        upper_penalty = torch.relu(predictions-self.max_value)
        penalty = torch.mean(upper_penalty)
        loss = rmse_loss + self.penalty_weight*penalty
        return loss
    
def log_grads(model, writer, epoch):
    for name, param in model.named_parameters():
        if param.requires_grad and "bias" not in name:
            writer.add_scalar(f"Gradients/{name}", param.grad.abs().mean(), epoch)

def get_optimizer(model,l2):
    # Set different learning rates for the transformer backbone and the decoder
    parameters = [
        {"params": model.transformer.parameters(), "lr": 1e-5},
        {"params": model.proj1.parameters(), "lr": 1e-5},
        {"params": model.head.parameters(), "lr": 1e-4},
        {"params": model.fusionblock1.parameters(), "lr": 1e-4},
        {"params": model.fusionblock2.parameters(), "lr": 1e-4},
        {"params": model.fusionblock3.parameters(), "lr": 1e-4},
        {"params": model.fusionblock4.parameters(), "lr": 1e-4},
        {"params": model.resample1.parameters(), "lr": 1e-4},
        {"params": model.resample2.parameters(), "lr": 1e-4},
        {"params": model.resample3.parameters(), "lr": 1e-4},
        {"params": model.resample4.parameters(), "lr": 1e-4},
    ]   
    
    optimizer = torch.optim.Adam(parameters,weight_decay=l2)
    return optimizer

def get_optimizer(model,l2):
    # Set different learning rates for the transformer backbone and the decoder
    parameters = [
        {"params": model.enc_conv1.parameters(), "lr": 1e-5},
        {"params": model.enc_conv2.parameters(), "lr": 1e-5},
        {"params": model.enc_conv3.parameters(), "lr": 1e-4},
        {"params": model.enc_conv4.parameters(), "lr": 1e-5},
        {"params": model.bottleneck.parameters(), "lr": 1e-5},
        {"params": model.upconv4.parameters(), "lr": 1e-4},
        {"params": model.dec_conv4.parameters(), "lr": 1e-4},
        {"params": model.upconv3.parameters(), "lr": 1e-4},
        {"params": model.dec_conv3.parameters(), "lr": 1e-4},
        {"params": model.upconv2.parameters(), "lr": 1e-4},
        {"params": model.dec_conv2.parameters(), "lr": 1e-4},
        {"params": model.upconv1.parameters(), "lr": 1e-4},
        {"params": model.dec_conv1.parameters(), "lr": 1e-4},
        {"params": model.output.parameters(), "lr": 1e-4},
    ]   
    
    optimizer = torch.optim.Adam(parameters,weight_decay=l2)
    return optimizer



def train_val(load, optim_split, loss_f, epochs, x_train, y_train, X_val, y_val, data_size, batch_size, model, lr, model_type, file_name, l2=True, dropout_bool=False, hidden_layers= 0, nheads=0, encoder_layers=0,transform=None):
    
    device = torch.device("cuda")
    model = model.to(device)
    model.load_state_dict(torch.load(load, map_location=device))
    for name, param in model.named_parameters():
        print(f"Parameter: {name}")
        print(f"Shape: {param.shape}")
        print(f"Initial values:\n{param.data}")
        print("------------------")

    loss_function = loss_f.to(device)
    initial_lr = lr
    step_size = 10  # Number of epochs before reducing the learning rate
    gamma = 0.66
    
    if l2:
        if optim_split:
            optimizer = get_optimizer(model,1e-5)
            print("get_optimizer_chosen_l2")
        else:
            optimizer = optim.Adam(model.parameters(),lr=lr, weight_decay=lr/10)
    else:
        if optim_split:
            optimizer = get_optimizer(model,0)
            print("get_optimizer_chosen")
        else:
            optimizer = optim.Adam(model.parameters(),lr=lr, weight_decay=0)
            
    # Initialize the StepLR scheduler
    scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    
    avg_train_loss_per_epoch_list = []
    avg_test_loss_per_epoch_list = []
    
    testing_dataset = MyDataset(X_val,y_val)
    testing_dl = DataLoader(testing_dataset,batch_size=batch_size)
    
    #start timer
    start_time = time.time()
    for epoch in range(1,epochs+1):
        rng_state = torch.get_rng_state()
        training_dataset = MyDataset(x_train,y_train,transform,epoch,42)
        torch.manual_seed(epoch+100)
        training_dl = DataLoader(training_dataset,batch_size=batch_size, num_workers= 0, shuffle=True)
        torch.set_rng_state(rng_state)

        print(f"current iteration: {epoch} / {epochs}")
        running_train_loss = 0.0
        running_test_loss = 0.0
        
        # Setting model to training mode
        model.train()
        # Training loop
        for i , (images, depth_images) in enumerate(training_dl):
            images, depth_images = images.to(device), depth_images.to(device)
            optimizer.zero_grad()
            outputs = model(images).unsqueeze(1)
            loss = loss_function(outputs,depth_images)
            batch_loss = loss.item()
            
            running_train_loss += batch_loss
            loss.backward()

                
            optimizer.step()
            
    
        
        current_lr = scheduler.get_last_lr()[0]
        print(f"Epoch {epoch}, Learning rate: {current_lr}")
        
        scheduler.step()    
        
        avg_train_loss = running_train_loss/len(training_dl)
        print("avg train loss",avg_train_loss)
        avg_train_loss_per_epoch_list.append(avg_train_loss)
    
        
        model.eval()
        # Validation / testing loop
        with torch.no_grad():
            for j, (images, depth_images) in enumerate(testing_dl):
                           
                # Moving images and depth images to GPU
                images,depth_images = images.to(device), depth_images.to(device)
                
                
                
                #forward pass
                outputs = model(images).unsqueeze(1)
                #print(torch.max(outputs))
                #print(torch.min(outputs))
                
                #loss = loss_function(outputs,depth_images)
                loss = loss_function(outputs,depth_images)
                #
                batch_loss = loss.item()
                
                running_test_loss += batch_loss
                
        avg_test_loss = running_test_loss/len(testing_dl)
        print("avg test loss", avg_test_loss)
        avg_test_loss_per_epoch_list.append(avg_test_loss)
        #print(avg_test_loss, f"epoch:{epoch}")
    
    #stop timer
    end_time = time.time()
    elapsed_time = end_time-start_time
    
    save_weights(model, model_type, epochs, file_name+".pt", data_size)
    make_csv(avg_train_loss_per_epoch_list,[avg_test_loss_per_epoch_list],
             ([data_size],[epoch],[batch_size],[lr],[l2],[dropout_bool],[hidden_layers],[nheads],[encoder_layers]),model_type, elapsed_time, file_name+".csv")
    
    
    return(avg_train_loss_per_epoch_list, avg_test_loss_per_epoch_list)



In [18]:
def full_test(load, optim_split, loss_f, epochs, x_train, y_train, X_test_VALID, y_test_VALID, X_test_MIDAIR, y_test_MIDAIR, data_size, batch_size, model, lr, model_type, file_name, l2=True, dropout_bool=False, hidden_layers= 0, nheads=0, encoder_layers=0,transform=None):

    
    device = torch.device("cuda")
    model = model.to(device)
    model.load_state_dict(torch.load(load, map_location=device))
    for name, param in model.named_parameters():
        print(f"Parameter: {name}")
        print(f"Shape: {param.shape}")
        print(f"Initial values:\n{param.data}")
        print("------------------")

    loss_function = loss_f.to(device)
    initial_lr = lr
    step_size = 10  # Number of epochs before reducing the learning rate
    gamma = 0.66
    
    if l2:
        if optim_split:
            optimizer = get_optimizer(model,1e-5)
            print("get_optimizer_chosen_l2")
        else:
            optimizer = optim.Adam(model.parameters(),lr=lr, weight_decay=lr/10)
    else:
        if optim_split:
            optimizer = get_optimizer(model,0)
            print("get_optimizer_chosen")
        else:
            optimizer = optim.Adam(model.parameters(),lr=lr, weight_decay=0)
            
    # Initialize the StepLR scheduler
    scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    
    avg_train_loss_per_epoch_list = []
    
    testing_dataset_VALID = MyDataset(X_test_VALID,y_test_VALID, None,0,42)
    testing_dl_VALID = DataLoader(testing_dataset_VALID,batch_size=16)
    
    testing_dataset_MIDAIR = MyDataset(X_test_MIDAIR,y_test_MIDAIR,None,0,42)
    testing_dl_MIDAIR = DataLoader(testing_dataset_MIDAIR,batch_size=16)
    
    VALID_rmse_list = []
    VALID_delta1_list, VALID_delta2_list, VALID_delta3_list = [],[],[]
    VALID_mae_list, VALID_absrel_list = [],[]
    MIDAIR_rmse_list = []
    MIDAIR_delta1_list, MIDAIR_delta2_list, MIDAIR_delta3_list = [],[],[]
    MIDAIR_mae_list, MIDAIR_absrel_list = [],[]

    
    #start timer
    start_time = time.time()
    for epoch in range(1,epochs+1):
        rng_state = torch.get_rng_state()
        training_dataset = MyDataset(x_train,y_train,transform,epoch,42)
        torch.manual_seed(epoch+100)
        training_dl = DataLoader(training_dataset,batch_size=batch_size, num_workers= 0, shuffle=True)
        torch.set_rng_state(rng_state)

        print(f"current iteration: {epoch} / {epochs}")
        running_train_loss = 0.0
        running_test_loss = 0.0
        
        # Setting model to training mode
        model.train()
        # Training loop
        for i , (images, depth_images) in enumerate(training_dl):
            images, depth_images = images.to(device), depth_images.to(device)
            optimizer.zero_grad()
            outputs = model(images).unsqueeze(1)
            #print(outputs.shape)
            loss = loss_function(outputs,depth_images)
            batch_loss = loss.item()
            
            running_train_loss += batch_loss
            loss.backward()

                
            optimizer.step()
            
        
        current_lr = scheduler.get_last_lr()[0]
        print(f"Epoch {epoch}, Learning rate: {current_lr}")
        
        scheduler.step()    
        
        avg_train_loss = running_train_loss/len(training_dl)
        print("avg train loss",avg_train_loss)
        avg_train_loss_per_epoch_list.append(avg_train_loss)
        
        

        
        
        VALID_rmse = 0
        VALID_delta1, VALID_delta2, VALID_delta3 = 0, 0, 0
        VALID_mae, VALID_absrel = 0, 0
        
        
        #Setting model to evaluation mode
        model.eval()
        # Validation / testing loop
        with torch.no_grad():
            for j, (images, depth_images) in enumerate(testing_dl_VALID):
    
                # Moving images and depth images to GPU
                images,depth_images = images.to(device), depth_images.to(device)
                
                #forward pass
                outputs = model(images).unsqueeze(1)
    
                # Calculating loss
                loss = WeightedRMSELoss1x()(outputs,depth_images)
                batch_loss = loss.item()

                VALID_rmse += batch_loss

                #Calculate MAE
                VALID_mae += torch.abs(outputs - depth_images).mean()

                #Calculate AbsRel
                depth_images[depth_images==-0.0008] = 0.01
                
                    
  
                VALID_absrel += torch.mean(torch.abs(outputs - depth_images) / depth_images)
                #print(VALID_absrel)

                #Calculate Delta
                ratio = torch.max(outputs / depth_images, depth_images / outputs)
                VALID_delta1 += (ratio < 1.25).float().mean()
                VALID_delta2 += (ratio < 1.25**2).float().mean()
                VALID_delta3 += (ratio < 1.25**3).float().mean()

        VALID_rmse_list.append(VALID_rmse/len(testing_dl_VALID))
        VALID_mae_list.append(VALID_mae.item()/len(testing_dl_VALID))
        VALID_absrel_list.append(VALID_absrel.item()/len(testing_dl_VALID))
        VALID_delta1_list.append(VALID_delta1.item()/len(testing_dl_VALID))
        VALID_delta2_list.append(VALID_delta2.item()/len(testing_dl_VALID))
        VALID_delta3_list.append(VALID_delta3.item()/len(testing_dl_VALID))
        
    
        MIDAIR_rmse = 0
        MIDAIR_delta1, MIDAIR_delta2, MIDAIR_delta3 = 0, 0, 0
        MIDAIR_mae, MIDAIR_absrel = 0, 0

        with torch.no_grad():
            for j, (images, depth_images) in enumerate(testing_dl_MIDAIR):

                # Moving images and depth images to GPU
                images,depth_images = images.to(device), depth_images.to(device)



                #forward pass
                outputs = model(images).unsqueeze(1)
                # Calculating loss
                loss = WeightedRMSELoss1x()(outputs,depth_images)
                batch_loss = loss.item()

                MIDAIR_rmse += batch_loss


                #Calculate MAE
                MIDAIR_mae += torch.abs(outputs - depth_images).mean()

                #Calculate AbsRel
                MIDAIR_absrel += torch.mean(torch.abs(outputs - depth_images) / depth_images)

        
                #Calculate Delta
                ratio = torch.max(outputs / depth_images, depth_images / outputs)
                #print(ratio)
                MIDAIR_delta1 += (ratio < 1.25).float().mean()
                MIDAIR_delta2 += (ratio < 1.25**2).float().mean()
                MIDAIR_delta3 += (ratio < 1.25**3).float().mean()
        #print(VALID_absrel.item()/len(testing_dl_MIDAIR))
        MIDAIR_rmse_list.append(MIDAIR_rmse/len(testing_dl_MIDAIR))
        MIDAIR_mae_list.append(MIDAIR_mae.item()/len(testing_dl_MIDAIR))
        MIDAIR_absrel_list.append(MIDAIR_absrel.item()/len(testing_dl_MIDAIR))
        MIDAIR_delta1_list.append(MIDAIR_delta1.item()/len(testing_dl_MIDAIR))
        MIDAIR_delta2_list.append(MIDAIR_delta2.item()/len(testing_dl_MIDAIR))
        MIDAIR_delta3_list.append(MIDAIR_delta3.item()/len(testing_dl_MIDAIR))
    #print(MIDAIR_rmse_list)
       #stop timer
    end_time = time.time()
    elapsed_time = end_time-start_time
    
    save_weights(model, model_type, epochs, file_name+".pt", data_size)
    make_csv(True,[epochs],[VALID_rmse_list,MIDAIR_rmse_list],
             (VALID_absrel_list,VALID_mae_list,VALID_delta1_list,VALID_delta2_list,VALID_delta3_list,MIDAIR_absrel_list,MIDAIR_mae_list,MIDAIR_delta1_list,MIDAIR_delta2_list,MIDAIR_delta3_list), model_type, elapsed_time, file_name+".csv")
    
    
    #return(avg_train_loss_per_epoch_list, avg_test_loss_per_epoch_list)
#def full_test( l2=True, dropout_bool=False, hidden_layers= 0, nheads=0, encoder_layers=0,transform=None):
#run = full_test("vit_fusion_4_heads_start_weights.pt",False,nn.MSELoss(),50, X_train, y_train, X_test_VALID, y_test_VALID, X_test_MIDAIR,y_test_MIDAIR, "extreme", 16, ViT_fusion_4_heads(), 0.00005, "vit", "vit_mse", False, False, 512,4,16,transform_no_crop)

In [19]:
#device = torch.device("cuda")
#vit = ViT_fusion_4_heads().to(device)
#vit.load_state_dict(torch.load(r"50epochs\vits\weights\extreme\vit_fusion_extreme_4_heads_TEST.pt"))

#vit2 = ViT_fusion_16_heads().to(device)
#vit2.load_state_dict(torch.load(r"50epochs\vits\weights\extreme\vit_fusion_scheduled_extreme_16_heads_TEST.pt"))
#cnn = CNN().to(device)

#cnn.load_state_dict(torch.load(r"50epochs\cnns\weights\extreme\cnn_extreme_TEST.pt"))

In [20]:
data = get_data("small", choice4)
#training_data = MyDataset(data[0][0].float(),data[0][1].float(),transform=transform) 
X_train, y_train = data[0][0].float(), data[0][1].float()
X_val, y_val = data[1][0].float(), data[1][1].float()
X_test_VALID, y_test_VALID = data[2][0].float(), data[2][1].float()
X_test_MIDAIR, y_test_MIDAIR = data[3][0].float(), data[3][1].float()

In [22]:
def find_distribution(X_test_VALID, y_test_VALID, X_test_MIDAIR, y_test_MIDAIR):
    testing_dataset_VALID = MyDataset(X_test_VALID, y_test_VALID, None, 0, 42)
    testing_dl_VALID = DataLoader(testing_dataset_VALID, batch_size=1)

    testing_dataset_MIDAIR = MyDataset(X_test_MIDAIR, y_test_MIDAIR, None, 0, 42)
    testing_dl_MIDAIR = DataLoader(testing_dataset_MIDAIR, batch_size=1)
    
    device = torch.device("cuda")
    
    valid = []
    midair = []
    d = []
    
    with torch.no_grad():
        for idx, (image, depth) in enumerate(testing_dl_VALID):
            image, depth = image.to(device), depth.to(device)
            depth = depth.squeeze(0).squeeze(0)
            depth = depth.flatten()
            valid.append(depth)    
            d.append(depth)

        for idx, (image, depth) in enumerate(testing_dl_MIDAIR):
            image, depth = image.to(device), depth.to(device)
            depth = depth.squeeze(0).squeeze(0)
            depth = depth.flatten()
            midair.append(depth)   
            d.append(depth)
            



    return valid, midair, d

d = find_distribution(X_test_VALID, y_test_VALID, X_test_MIDAIR, y_test_MIDAIR)

In [27]:
"""data1 = torch.stack(d[0])
data1 = data1.view(-1)
subsample_indices1 = torch.randint(0, len(data1), (len(data1),))
subsample1 = data1[subsample_indices1]
# Convert the subsample to a NumPy array for plotting
subsample1 = subsample1.detach().cpu().numpy()
# Define the number of bins for your histogram
num_bins = 50
# Create and display the histogram
plt.hist(reverse_norm(subsample1), bins=num_bins,color="blue")
plt.title("Distribution of Depth values in VALID")
plt.ylabel("Number of Pixels")
plt.xlabel("Depth value")
plt.savefig("valid_depth_distribution")
plt.show()"""

'data1 = torch.stack(d[0])\ndata1 = data1.view(-1)\nsubsample_indices1 = torch.randint(0, len(data1), (len(data1),))\nsubsample1 = data1[subsample_indices1]\n# Convert the subsample to a NumPy array for plotting\nsubsample1 = subsample1.detach().cpu().numpy()\n# Define the number of bins for your histogram\nnum_bins = 50\n# Create and display the histogram\nplt.hist(reverse_norm(subsample1), bins=num_bins,color="blue")\nplt.title("Distribution of Depth values in VALID")\nplt.ylabel("Number of Pixels")\nplt.xlabel("Depth value")\nplt.savefig("valid_depth_distribution")\nplt.show()'

In [28]:
"""data2 = torch.stack(d[1])
data2 = data2.view(-1)
subsample_indices2 = torch.randint(0, len(data2), (len(data2),))
subsample2 = data2[subsample_indices2]
# Convert the subsample to a NumPy array for plotting
subsample2 = subsample2.detach().cpu().numpy()
# Define the number of bins for your histogram
num_bins = 50
# Create and display the histogram
plt.hist(reverse_norm(subsample2), bins=num_bins,color="green")
#plt.hist(reverse_norm(subsample1), bins=num_bins,color="blue",alpha=0.5)
plt.title("Distribution of Depth values in Mid-Air")
plt.ylabel("Number of Pixels")
plt.xlabel("Depth value")
plt.savefig("midair_depth_distribution")
plt.show()"""

'data2 = torch.stack(d[1])\ndata2 = data2.view(-1)\nsubsample_indices2 = torch.randint(0, len(data2), (len(data2),))\nsubsample2 = data2[subsample_indices2]\n# Convert the subsample to a NumPy array for plotting\nsubsample2 = subsample2.detach().cpu().numpy()\n# Define the number of bins for your histogram\nnum_bins = 50\n# Create and display the histogram\nplt.hist(reverse_norm(subsample2), bins=num_bins,color="green")\n#plt.hist(reverse_norm(subsample1), bins=num_bins,color="blue",alpha=0.5)\nplt.title("Distribution of Depth values in Mid-Air")\nplt.ylabel("Number of Pixels")\nplt.xlabel("Depth value")\nplt.savefig("midair_depth_distribution")\nplt.show()'

In [29]:
"""data3 = torch.stack(d[2])
data3 = data3.view(-1)
subsample_indices3 = torch.randint(0, len(data3), (len(data3),))
subsample3 = data3[subsample_indices3]
# Convert the subsample to a NumPy array for plotting
subsample3 = subsample3.detach().cpu().numpy()
# Define the number of bins for your histogram
num_bins = 50
# Create and display the histogram
plt.hist(reverse_norm(subsample3), bins=num_bins,color="orange")
plt.title("Depth Values from Combined dataset")
plt.ylabel("Number of Pixels")
plt.xlabel("Depth value")
plt.savefig("combined_depth_distribution")
plt.show()"""

'data3 = torch.stack(d[2])\ndata3 = data3.view(-1)\nsubsample_indices3 = torch.randint(0, len(data3), (len(data3),))\nsubsample3 = data3[subsample_indices3]\n# Convert the subsample to a NumPy array for plotting\nsubsample3 = subsample3.detach().cpu().numpy()\n# Define the number of bins for your histogram\nnum_bins = 50\n# Create and display the histogram\nplt.hist(reverse_norm(subsample3), bins=num_bins,color="orange")\nplt.title("Depth Values from Combined dataset")\nplt.ylabel("Number of Pixels")\nplt.xlabel("Depth value")\nplt.savefig("combined_depth_distribution")\nplt.show()'

In [30]:
"""# Set the figure size
plt.figure(figsize=(16, 12))
# Plot red and blue histograms together
plt.subplot(2, 2, 1)
plt.hist(reverse_norm(subsample1), bins=50, alpha=0.5, color='b', label='VALID')
plt.hist(reverse_norm(subsample2), bins=50, alpha=0.5, color='g', label='Mid-Air')
plt.title("Depth Values plotted together")
plt.ylabel("Number of Pixels")
plt.xlabel("Depth value")
plt.legend()

plt.subplot(2, 2, 2)
plt.hist(reverse_norm(subsample3), bins=num_bins,color="orange")
plt.title("Depth Values from Combined dataset")
plt.ylabel("Number of Pixels")
plt.xlabel("Depth value")
plt.subplots_adjust(bottom=0.1)
plt.tight_layout()
plt.savefig("all_distributions", bbox_inches='tight')

plt.show()

"""

'# Set the figure size\nplt.figure(figsize=(16, 12))\n# Plot red and blue histograms together\nplt.subplot(2, 2, 1)\nplt.hist(reverse_norm(subsample1), bins=50, alpha=0.5, color=\'b\', label=\'VALID\')\nplt.hist(reverse_norm(subsample2), bins=50, alpha=0.5, color=\'g\', label=\'Mid-Air\')\nplt.title("Depth Values plotted together")\nplt.ylabel("Number of Pixels")\nplt.xlabel("Depth value")\nplt.legend()\n\nplt.subplot(2, 2, 2)\nplt.hist(reverse_norm(subsample3), bins=num_bins,color="orange")\nplt.title("Depth Values from Combined dataset")\nplt.ylabel("Number of Pixels")\nplt.xlabel("Depth value")\nplt.subplots_adjust(bottom=0.1)\nplt.tight_layout()\nplt.savefig("all_distributions", bbox_inches=\'tight\')\n\nplt.show()\n\n'

In [31]:
"""# Now you can plot your losses
plt.figure(figsize=(20, 12))
#vit_valid_losses_new, cnn_valid_losses_new, vit_midair_losses_new, cnn_midair_losses_new
plt.subplot(2, 2, 1)
#plt.hist(vit_valid_losses, bins=50, alpha=0.5, color='r', label='ViT_VALID')
plt.hist([reverse_norm(loss) for loss in vit_valid_losses], bins=50, alpha=0.5, color='r', label='ViT_VALID')
plt.xlabel('Loss in meters')
plt.ylabel('Count')
plt.legend()

plt.subplot(2, 2, 2)
#plt.hist(vit2_valid_losses, bins=50, alpha=0.5, color='b', label='ViT2_VALID')
plt.hist([reverse_norm(loss) for loss in vit2_valid_losses], bins=50, alpha=0.5, color='b', label='vit2_VALID')
plt.xlabel('Loss in meters')
plt.ylabel('Count')
plt.legend()

plt.subplot(2, 2, 3)
#plt.hist(vit_midair_losses, bins=50, alpha=0.5, color='g', label='ViT_Mid-Air')
plt.hist([reverse_norm(loss) for loss in vit_midair_losses], bins=50, alpha=0.5, color='g', label='ViT_Mid-Air')
plt.xlabel('Loss in meters')
plt.ylabel('Count')
plt.legend()

plt.subplot(2, 2, 4)
#plt.hist(vit2_midair_losses, bins=50, alpha=0.5, color='y', label='ViT2_Mid-Air')
plt.hist([reverse_norm(loss) for loss in vit2_midair_losses], bins=50, alpha=0.5, color='y', label='vit2_Mid-Air')
plt.xlabel('Loss in meters')
plt.ylabel('Count')
plt.legend()

plt.tight_layout()


# Save the plot as a PNG file
#plt.savefig('loss_distribution.png', dpi=300, format='png')
plt.show()"""

"# Now you can plot your losses\nplt.figure(figsize=(20, 12))\n#vit_valid_losses_new, cnn_valid_losses_new, vit_midair_losses_new, cnn_midair_losses_new\nplt.subplot(2, 2, 1)\n#plt.hist(vit_valid_losses, bins=50, alpha=0.5, color='r', label='ViT_VALID')\nplt.hist([reverse_norm(loss) for loss in vit_valid_losses], bins=50, alpha=0.5, color='r', label='ViT_VALID')\nplt.xlabel('Loss in meters')\nplt.ylabel('Count')\nplt.legend()\n\nplt.subplot(2, 2, 2)\n#plt.hist(vit2_valid_losses, bins=50, alpha=0.5, color='b', label='ViT2_VALID')\nplt.hist([reverse_norm(loss) for loss in vit2_valid_losses], bins=50, alpha=0.5, color='b', label='vit2_VALID')\nplt.xlabel('Loss in meters')\nplt.ylabel('Count')\nplt.legend()\n\nplt.subplot(2, 2, 3)\n#plt.hist(vit_midair_losses, bins=50, alpha=0.5, color='g', label='ViT_Mid-Air')\nplt.hist([reverse_norm(loss) for loss in vit_midair_losses], bins=50, alpha=0.5, color='g', label='ViT_Mid-Air')\nplt.xlabel('Loss in meters')\nplt.ylabel('Count')\nplt.legend()\n\

In [32]:
"""# Set the figure size
plt.figure(figsize=(12, 10))

# Plot red and blue histograms together
plt.subplot(2, 2, 1)
#plt.hist([reverse_norm(x) for x in vit_valid_no_extreme_outliers], bins=100, alpha=0.5, color='r', label='ViT_VALID')
plt.hist([reverse_norm(x) for x in vit_valid_losses], bins=100, alpha=0.5, color='r', label='ViT_4_heads VALID')
plt.hist([reverse_norm(x) for x in vit2_valid_losses], bins=100, alpha=0.5, color='b', label='ViT_16_heads VALID')
plt.xlabel('Loss in meters')
plt.ylabel('Count')
plt.legend()

# Plot red and yellow histograms together
plt.subplot(2, 2, 2)
plt.hist([reverse_norm(x) for x in vit_midair_losses], bins=100, alpha=0.5, color='orange', label='ViT_4_heads Mid-Air')
plt.hist([reverse_norm(x) for x in vit2_midair_losses], bins=100, alpha=0.5, color='g', label='ViT_16_heads Mid-Air')
plt.xlabel('Loss in meters')
plt.ylabel('Count')
plt.legend()

# Add a title to your plot
#plt.suptitle('Loss Distributions with 9 extreme outliers removed from ViT_VALID')

#plt.tight_layout()

# Save the plot as a PNG file
plt.savefig('loss_distribution_vit_valid_removed_outliers_4_16.png', dpi=300, format='png', bbox_inches='tight', pad_inches=0)

plt.show()"""

"# Set the figure size\nplt.figure(figsize=(12, 10))\n\n# Plot red and blue histograms together\nplt.subplot(2, 2, 1)\n#plt.hist([reverse_norm(x) for x in vit_valid_no_extreme_outliers], bins=100, alpha=0.5, color='r', label='ViT_VALID')\nplt.hist([reverse_norm(x) for x in vit_valid_losses], bins=100, alpha=0.5, color='r', label='ViT_4_heads VALID')\nplt.hist([reverse_norm(x) for x in vit2_valid_losses], bins=100, alpha=0.5, color='b', label='ViT_16_heads VALID')\nplt.xlabel('Loss in meters')\nplt.ylabel('Count')\nplt.legend()\n\n# Plot red and yellow histograms together\nplt.subplot(2, 2, 2)\nplt.hist([reverse_norm(x) for x in vit_midair_losses], bins=100, alpha=0.5, color='orange', label='ViT_4_heads Mid-Air')\nplt.hist([reverse_norm(x) for x in vit2_midair_losses], bins=100, alpha=0.5, color='g', label='ViT_16_heads Mid-Air')\nplt.xlabel('Loss in meters')\nplt.ylabel('Count')\nplt.legend()\n\n# Add a title to your plot\n#plt.suptitle('Loss Distributions with 9 extreme outliers remov

In [33]:
def show_index_2(vit_model, cnn_model, choose, index, X_test_VALID, y_test_VALID, X_test_MIDAIR, y_test_midair):
    nrows = 1
    ncolumns = 4
    subplot_width = 2.7
    subplot_height = 2.7 
    extra_width_space = 0.5
    extra_height_space = 0.5
    fig_width = ncolumns*subplot_width + extra_width_space
    fig_height = nrows*subplot_height + extra_height_space
    fig_size = (fig_width, fig_height)
    
    if choose.lower() == "valid":
        testing_dataset = MyDataset(X_test_VALID,y_test_VALID, None,0, 42)
        testing_dl = DataLoader(testing_dataset,batch_size=1)
    else:
        testing_dataset = MyDataset(X_test_MIDAIR,y_test_MIDAIR,None,0,42)
        testing_dl = DataLoader(testing_dataset,batch_size=1)
    
    fig1, axes1 = plt.subplots(nrows,ncolumns, figsize=fig_size)
    fig1.subplots_adjust(wspace=0.2, hspace=0.2)
    vit_model.eval()
    cnn_model.eval()
    loss_function = WeightedRMSELoss1x().to(device)
    with torch.no_grad():
        for idx, (image, depth) in enumerate(Subset(testing_dataset,[index])):   
            image, depth = image.to(device), depth.to(device)
            image = image.unsqueeze(0)
            vit_depth_pred = vit_model(image).squeeze(0) 
            cnn_depth_pred = cnn_model(image).squeeze(0)
            print("vit loss", loss_function(vit_depth_pred,depth))
            print("cnn loss", loss_function(cnn_depth_pred,depth))

            image[:,0,:,:] = image[:,0,:,:]*72.5+79.88429260253906
            image[:,1,:,:] = image[:,1,:,:]*67.4375+112.80674743652344
            image[:,2,:,:] = image[:,2,:,:]*72.4375+109.85155487060547

            image_normalized = (image.cpu().squeeze(0))/287
            cbar_ax = fig1.add_axes([0.12, 0.1, 0.785, 0.05]) 
            
            # for original image
            axes1[0].imshow(image_normalized.squeeze(0).permute(1,2,0))
            axes1[0].set_title("Original_Image")
            axes1[0].axis("off")

            # for ViT prediction
            img2 = axes1[1].imshow(reverse_norm(vit_depth_pred.cpu().numpy()), vmin=0, vmax=reverse_norm(1))
            axes1[1].set_title("ViT_Prediction (4 heads)")
            axes1[1].axis("off")

            # for CNN prediction
            img3 = axes1[2].imshow(reverse_norm(cnn_depth_pred.cpu().numpy()), vmin=0, vmax=reverse_norm(1))
            axes1[2].set_title("ViT_Prediction (16 heads)")
            axes1[2].axis("off")

            # for ground truth
            img4 = axes1[3].imshow(reverse_norm(depth.squeeze(0).squeeze(0).cpu().numpy()), vmin=0, vmax=reverse_norm(1))
            axes1[3].set_title("Ground_Truth")
            axes1[3].axis("off")

            # Single colorbar for all subplots
            cbar = fig1.colorbar(img4, cax=cbar_ax, orientation='horizontal')
            cbar.ax.set_xticklabels(['{:.1f} m'.format(i) for i in cbar.get_ticks()])

    fig1.show()
    
    #fig1.savefig("midair_7_vit_superior.png", bbox_inches="tight")  # Save the figure with tight bounding box
#for i in range(500):
    
show_index_2(vit,vit2,"midair",1,X_test_VALID, y_test_VALID, X_test_MIDAIR, y_test_MIDAIR)   

NameError: name 'vit' is not defined

In [34]:
def predictions(vit_model, cnn_model, nrows, X_test_VALID, y_test_VALID, X_test_MIDAIR, y_test_midair):
    nrows = nrows
    ncolumns = 4
    subplot_width = 2.7
    subplot_height = 2.7 
    extra_width_space = 0.5
    extra_height_space = 0.5
    
    fig_width = ncolumns*subplot_width + extra_width_space
    fig_height = nrows*subplot_height + extra_height_space
    fig_size = (fig_width, fig_height)

    
    
    
    
    testing_dataset_VALID = MyDataset(X_test_VALID,y_test_VALID, None,0, 42)
    testing_dl_VALID = DataLoader(testing_dataset_VALID,batch_size=1)
    
    testing_dataset_MIDAIR = MyDataset(X_test_MIDAIR,y_test_MIDAIR,None,0,42)
    testing_dl_MIDAIR = DataLoader(testing_dataset_MIDAIR,batch_size=1)

    
    
    fig1, axes1 = plt.subplots(nrows,ncolumns, figsize=fig_size)
    fig2, axes2 = plt.subplots(nrows,ncolumns, figsize=fig_size)
    
    
    fig1.subplots_adjust(wspace=0.2, hspace=0.2)
    #loss = nn.
    loss_function = WeightedRMSELoss1x().to(device)
    #MIDAIR LOOP
    vit_model.eval()
    cnn_model.eval()

    with torch.no_grad():
        
        for idx, (image, depth) in enumerate(testing_dl_MIDAIR):   
            image, depth = image.to(device), depth.to(device)
            if idx == nrows:
                break

            vit_depth_pred = vit_model(image).squeeze(0) 
            cnn_depth_pred = cnn_model(image).squeeze(0)

            
            image[:,0,:,:] = image[:,0,:,:]*72.5+79.88429260253906
            image[:,1,:,:] = image[:,1,:,:]*67.4375+112.80674743652344
            image[:,2,:,:] = image[:,2,:,:]*72.4375+109.85155487060547
            

            image_normalized = (image.cpu().squeeze(0))/287
            
            axes1[idx,0].imshow(image_normalized.squeeze(0).permute(1,2,0))
            axes1[idx,0].set_title(f"Original_Image{idx}")
            axes1[idx,0].axis("off")
            
            
            img2 = axes1[idx,1].imshow(reverse_norm(vit_depth_pred.cpu().numpy()), vmin=0, vmax=reverse_norm(1))
            axes1[idx,1].set_title("ViT_Prediction")
            axes1[idx,1].axis("off")
            fig1.colorbar(img2,ax=axes1[idx,1])
            
        
            img3 = axes1[idx,2].imshow(reverse_norm(cnn_depth_pred.cpu().numpy()), vmin=0, vmax=reverse_norm(1))
            axes1[idx,2].set_title("CNN_Prediction")
            axes1[idx,2].axis("off")
            fig1.colorbar(img3,ax=axes1[idx,2])
            
            img4 = axes1[idx,3].imshow(reverse_norm(depth.squeeze(0).squeeze(0).cpu().numpy()), vmin=0, vmax=reverse_norm(1))
            axes1[idx,3].set_title("Ground_Truth")
            axes1[idx,3].axis("off")
            fig1.colorbar(img4,ax=axes1[idx,3])
       
    fig1.show()
    
    with torch.no_grad():
        
        for idx, (image, depth) in enumerate(testing_dl_VALID):   
            image, depth = image.to(device), depth.to(device)
            if idx == nrows:
                break

            vit_depth_pred = vit_model(image).squeeze(0)   
            cnn_depth_pred = cnn_model(image).squeeze(0)

            image[:,0,:,:] = image[:,0,:,:]*72.5+79.88429260253906
            image[:,1,:,:] = image[:,1,:,:]*67.4375+112.80674743652344
            image[:,2,:,:] = image[:,2,:,:]*72.4375+109.85155487060547
            
            image_normalized = (image.cpu().squeeze(0))/287
            
            axes2[idx,0].imshow(image_normalized.squeeze(0).permute(1,2,0))
            axes2[idx,0].set_title(f"Original_Image{idx}")
            axes2[idx,0].axis("off")
            
            
            img2 = axes2[idx,1].imshow(reverse_norm(vit_depth_pred.cpu().numpy()), vmin=0, vmax=reverse_norm(1))
            axes2[idx,1].set_title("ViT_Prediction")
            axes2[idx,1].axis("off")
            fig1.colorbar(img2,ax=axes2[idx,1])
            
        
            img3 = axes2[idx,2].imshow(reverse_norm(cnn_depth_pred.cpu().numpy()), vmin=0, vmax=reverse_norm(1))
            axes2[idx,2].set_title("CNN_Prediction")
            axes2[idx,2].axis("off")
            fig1.colorbar(img3,ax=axes2[idx,2])
            
            img4 = axes2[idx,3].imshow(reverse_norm(depth.squeeze(0).squeeze(0).cpu().numpy()), vmin=0, vmax=reverse_norm(1))
            axes2[idx,3].set_title("Ground_Truth")
            axes2[idx,3].axis("off")
            fig1.colorbar(img4,ax=axes2[idx,3])
       
    fig2.show()
device = torch.device("cuda")  
vit = ViT_fusion_4_heads().to(device)
vit.load_state_dict(torch.load(r"50epochs\vits\weights\tiny\vit_fusion_tiny_4_heads_TEST.pt"))
cnn = CNN().to(device)
cnn.load_state_dict(torch.load(r"50epochs\cnns\weights\tiny\cnn_tiny_TEST.pt"))
predictions(vit,cnn,1,X_test_VALID, y_test_VALID, X_test_MIDAIR, y_test_MIDAIR)         

NameError: name 'Rearrange' is not defined

In [64]:
def find_over_1_normalized(load, optim_split, loss_f, epochs, x_train, y_train, X_test_VALID, y_test_VALID, X_test_MIDAIR, y_test_MIDAIR, data_size, batch_size, model, lr, model_type, file_name, l2=True, dropout_bool=False, hidden_layers= 0, nheads=0, encoder_layers=0,transform=None):

    
    device = torch.device("cuda")
    model = model.to(device)
    model.load_state_dict(torch.load(load, map_location=device))
    #for name, param in model.named_parameters():
        #print(f"Parameter: {name}")
        #print(f"Shape: {param.shape}")
        #print(f"Initial values:\n{param.data}")
        #print("------------------")

    loss_function = loss_f.to(device)
    initial_lr = lr
    step_size = 10  # Number of epochs before reducing the learning rate
    gamma = 0.66
    
    if l2:
        if optim_split:
            optimizer = get_optimizer(model,1e-5)
            print("get_optimizer_chosen_l2")
        else:
            optimizer = optim.Adam(model.parameters(),lr=lr, weight_decay=lr/10)
    else:
        if optim_split:
            optimizer = get_optimizer(model,0)
            print("get_optimizer_chosen")
        else:
            optimizer = optim.Adam(model.parameters(),lr=lr, weight_decay=0)
            
    # Initialize the StepLR scheduler
    scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    
    avg_train_loss_per_epoch_list = []
    
    testing_dataset_VALID = MyDataset(X_test_VALID,y_test_VALID, None,0,42)
    testing_dl_VALID = DataLoader(testing_dataset_VALID,batch_size=16)
    
    testing_dataset_MIDAIR = MyDataset(X_test_MIDAIR,y_test_MIDAIR,None,0,42)
    testing_dl_MIDAIR = DataLoader(testing_dataset_MIDAIR,batch_size=16)
    
    VALID_rmse_list = []
    VALID_delta1_list, VALID_delta2_list, VALID_delta3_list = [],[],[]
    VALID_mae_list, VALID_absrel_list = [],[]
    MIDAIR_rmse_list = []
    MIDAIR_delta1_list, MIDAIR_delta2_list, MIDAIR_delta3_list = [],[],[]
    MIDAIR_mae_list, MIDAIR_absrel_list = [],[]

    
    #start timer
    start_time = time.time()
    
        

        

    VALID_rmse = 0
    VALID_delta1, VALID_delta2, VALID_delta3 = 0, 0, 0
    VALID_mae, VALID_absrel = 0, 0

    valid_over_1 = 0
    total_valid = 0
    
    midair_over_1 = 0
    total_midair = 0
    #Setting model to evaluation mode
    model.eval()
    # Validation / testing loop
    with torch.no_grad():
        for j, (images, depth_images) in enumerate(testing_dl_VALID):

            # Moving images and depth images to GPU
            images,depth_images = images.to(device), depth_images.to(device)

            #forward pass
            outputs = model(images).unsqueeze(1)

            mask = torch.where(depth_images==1,1,0)
            mask_sum = torch.sum(mask).item()
           
            valid_over_1 += mask_sum
            total_valid += mask.numel()

            
            
            
            # Calculating loss
            loss = WeightedRMSELoss1x()(outputs,depth_images)
            #print(loss)
            batch_loss = loss.item()

            VALID_rmse += batch_loss
    
    valid_r = valid_over_1/total_valid
    print(valid_over_1/total_valid)



            
    


    MIDAIR_rmse = 0
    MIDAIR_delta1, MIDAIR_delta2, MIDAIR_delta3 = 0, 0, 0
    MIDAIR_mae, MIDAIR_absrel = 0, 0

    with torch.no_grad():
        for j, (images, depth_images) in enumerate(testing_dl_MIDAIR):

            # Moving images and depth images to GPU
            images,depth_images = images.to(device), depth_images.to(device)


            

            #forward pass
            outputs = model(images).unsqueeze(1)

            mask = torch.where(depth_images==1,1,0)
            mask_sum = torch.sum(mask).item()
           
            midair_over_1 += mask_sum
            total_midair += mask.numel()

            
            # Calculating loss
            loss = WeightedRMSELoss1x()(outputs,depth_images)
            batch_loss = loss.item()

            MIDAIR_rmse += batch_loss

    midair_r = midair_over_1/total_midair
    print(midair_over_1/total_midair)
    print((valid_r*(0.1313)+(midair_r)*(1-0.1313)))
    print("\n")
    return valid_r,midair_r,(valid_r*(0.1313)+(midair_r)*(1-0.1313))
run_e = test2(r"50epochs\vits\weights\extreme\vit_fusion_extreme_4_heads_TEST.pt",False,WeightedRMSELoss1x(),50, X_train, y_train, X_test_VALID, y_test_VALID, X_test_MIDAIR,y_test_MIDAIR, "extreme", 16, ViT_fusion_4_heads(), 0.00005, "vit", "vit_mse", False, False, 512,4,16,transform_no_crop)

0.2408781070980049
0.40179636686158887
0.3806677993546303




In [116]:
"""r1 = np.random.randint(0,len(X_test_VALID),30)
r2 = np.random.randint(0,len(X_test_MIDAIR),30)
print(r1)"""

[241 250 876   4 118 800 373  64 145 223 238 176 778 852 281  62 216 853
 826 794 688 460 928 609 104  98 510 384 404 822]


In [36]:
"""r1 = np.random.randint(0,len(X_test_VALID),30)
# Use a for loop to create your 16 subplots
# Create a new figure
fig = plt.figure(figsize=(14, 16))
for i in range(30):
    ax = fig.add_subplot(6, 5, i+1)
    ax.imshow(np.transpose(X_test_VALID[r1[i]], (1,2,0)))
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    ax.axis('off')  # Turn off the axis labels for each subplot
    
fig.tight_layout()
fig.savefig("30-images-valid", bbox_inches='tight')"""

'r1 = np.random.randint(0,len(X_test_VALID),30)\n# Use a for loop to create your 16 subplots\n# Create a new figure\nfig = plt.figure(figsize=(14, 16))\nfor i in range(30):\n    ax = fig.add_subplot(6, 5, i+1)\n    ax.imshow(np.transpose(X_test_VALID[r1[i]], (1,2,0)))\n    plt.subplots_adjust(wspace=0.1, hspace=0.1)\n    ax.axis(\'off\')  # Turn off the axis labels for each subplot\n    \nfig.tight_layout()\nfig.savefig("30-images-valid", bbox_inches=\'tight\')'

In [35]:
"""fig1 = plt.figure(figsize=(14, 16))
for i in range(30):
    ax = fig1.add_subplot(6, 5, i+1)
    ax.imshow(np.transpose(X_test_MIDAIR[r1[i]], (1,2,0)))
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    ax.axis('off')  # Turn off the axis labels for each subplot
    
fig1.tight_layout()
fig1.savefig("30-images-midair", bbox_inches='tight')
"""

'fig1 = plt.figure(figsize=(14, 16))\nfor i in range(30):\n    ax = fig1.add_subplot(6, 5, i+1)\n    ax.imshow(np.transpose(X_test_MIDAIR[r1[i]], (1,2,0)))\n    plt.subplots_adjust(wspace=0.1, hspace=0.1)\n    ax.axis(\'off\')  # Turn off the axis labels for each subplot\n    \nfig1.tight_layout()\nfig1.savefig("30-images-midair", bbox_inches=\'tight\')\n'